Escalonamento

In [ ]:
import numpy as np
import copy

def escalona(M):
    n = len(M)
    for j in range(n-1):
        for i in range(j,n-1): 
            # rolando as linhas
            while M[j,j]==0:
                M[j:,:] = np.roll(M[j:,:], -1, axis=0)
            m = M[i+1,j]/M[j,j]
            M[i+1] = M[i+1]-m*M[j]
    return (M)

def solve_tril(L,b):
    y = np.empty(len(b))
    for i in range(len(b)): 
        y[i] = (b[i] - np.dot(L[i,0:i],y[0:i]))/L[i][i] 
    return y

# função para sistema triangular superior
def solve_triu(U,y):
    n=len(y)
    x = np.empty(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.sum(U[i,i+1:n+1]*x[i+1:n+1]))/U[i,i]
    return x

Fatoração LU

In [ ]:
def fatoraLU(A):
    L = np.eye(len(A))
    U = A.copy()
    n = len(A)
    for j in range(n-1):
        for i in range(j,n-1):            
            m = U[i+1,j]/U[j,j]
            U[i+1] = U[i+1]-m*U[j]
            L[i+1,j] = m 
    return (L,U)

# usando o algoritmo de doolitle
def fatoracao_LU(A):
    n = len(A)     
    U = np.zeros((n,n))
    L = np.identity(n)
    for m in range(n):
        for j in range(m, n):
            U[m,j] = A[m,j] - np.sum(L[m,0:m] * U[0:m,j])
        for i in range(m+1, n):
            L[i,m] = (A[i,m] - np.sum(L[i,0:m] * U[0:m,m]))/U[m,m]
    return L,U

Eliminação de Gauss

In [ ]:
def gaussElimin(a,b): 
    n = len(b) 
    for k in range(0,n-1): 
        for i in range(k+1,n): 
            if a[i][k] != 0.0: 
                lam = a [i][k]/a[k][k] 
                a[i][k+1:n] = a[i][k+1:n] - lam*a[k][k+1:n] 
                b[i] = b[i] - lam*b[k] 
    for k in range(n-1,-1,-1): 
        b[k] = (b[k] - np.dot(a[k][k+1:n],b[k+1:n]))/a[k][k] 
    return b


# GaussJordan
def gaussJordan(M):
    n = len(M)
    for k in range(n):
        M[k] = M[k]/M[k,k]
        for i in range(n):
            if i!=k:
                M[i] = M[i] - M[i,k]*M[k]
    return (M)

A = np.array([[2,1,3],[0,-1,1],[1,0,3]], float)
I = np.identity(len(A))
M = np.concatenate((A,I), axis=1)

print ('Matriz ampliada:')
print (M)

print ('Matriz após eliminação:')
Mgj = gaussJordan(M)
print (Mgj )

print ('Matriz inversa:')
Ainv = Mgj[:,3:6]
print(Ainv)

Método de Jacobi-Richardson

In [ ]:
from pprint import pprint
from numpy import array, zeros, diag, diagflat, dot

def jacobi(A,b,x=zeros(len(A)), eps=1e-5):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # Create a vector of the diagonal elements of A                                                                                                                                                
    # and subtract them from A
    err = 1
    x_ant = x
    D = diag(A)
    R = A - diagflat(D)

    # Iterate for N times                                                                                                                                                                          
    while err > eps:
        x = (b - dot(R,x)) / D
        err = abs(max(x-x_ant)/max(x))
        x_ant = x
    return x

Método de Gauss Seidel

In [ ]:
from numpy.linalg import inv

def gauss_seidel(A, b, x=np.zeros(len(A)), eps=1e-5):
    err = 1
    x_ant = x
    n = len(A)
    P = np.zeros((n,n)).astype(float)
    Q = np.zeros((n,n)).astype(float)
    g = np.zeros(n).astype(float)


    for i in range(n):
        P[i,0:i] = -A[i,0:i]/A[i,i]
        Q[i,i+1:n] = -A[i,i+1:n]/A[i,i]
        g[i] = b[i]/A[i,i] 

    I = np.eye(n)
    H = np.dot(inv(I-P),Q)
    g = np.dot(inv(I-P),g)

    
    # verificar convergência
#    beta = []
#    for i in range(len(H)):
#        bi = np.dot(beta,abs(H[i,0:i]))+ np.sum(abs(H[i,i+1:]))
#        beta.append(bi)
#    print(f'max(beta) ={np.array(beta).max()}')
    
    while err>eps:
        x = np.dot(H,x) + g
        err = abs(max(x-x_ant)/max(x))
        x_ant = x
        
    return x